In [ ]:
# | default_exp tools.code_quality

In [1]:
# | export
from agentic.tools.base import Tool
from agentic.schemas import RepoQualityAnalyzerParams, QualityAnalysisResponse
from pathlib import Path
import fnmatch
from typing import Dict, Any, List

class CodeQualityTool(Tool):
    def get_tool_schema(self) -> Dict[str, Any]:
        """Return the OpenAI-compatible schema for code_quality."""
        return {
                "type": "function",
                "function": {
                    "name": "code_quality",
                    "description": "Comprehensive repository code quality analysis with actionable recommendations for improvements.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "repo_path": {"type": "string", "description": "Repository root path"},
                            "analysis_type": {
                                "type": "string",
                                "enum": ["full", "security", "performance", "maintainability", "documentation"],
                                "description": "Type of analysis to perform"
                            },
                            "file_patterns": {"type": "array", "items": {"type": "string"}, "description": "File patterns to analyze (e.g., '*.py', '*.js')"},
                            "exclude_patterns": {"type": "array", "items": {"type": "string"}, "description": "Patterns to exclude"},
                            "severity_threshold": {"type": "string", "enum": ["low", "medium", "high", "critical"], "description": "Minimum severity for recommendations"}
                        },
                        "required": ["repo_path"]
                    }
                }
            }
    def execute(self, **kwargs) -> Dict[str, Any]:
        """Analyze code quality for a repository."""
        try:
            params = RepoQualityAnalyzerParams(**kwargs)
            repo_path = Path(params.repo_path)
            
            if not repo_path.exists() or not repo_path.is_dir():
                return {"error": f"Invalid repository path: {params.repo_path}"}
            
            file_patterns = params.file_patterns or ["*.py", "*.js", "*.java"]
            exclude_patterns = params.exclude_patterns or ["*.pyc", "__pycache__/*"]
            files_analyzed = []
            
            for root, _, files in os.walk(params.repo_path):
                for file in files:
                    file_path = Path(root) / file
                    if any(fnmatch.fnmatch(file, pattern) for pattern in file_patterns) and \
                       not any(fnmatch.fnmatch(str(file_path), pattern) for pattern in exclude_patterns):
                        files_analyzed.append(str(file_path))
            
            # Simplified quality analysis (replace with actual analysis tools like pylint)
            recommendations = [
                {"file": f, "issue": "Placeholder issue", "suggestion": "Placeholder suggestion"}
                for f in files_analyzed[:5]  # Limit for brevity
            ]
            
            return QualityAnalysisResponse(
                repository=params.repo_path,
                files_analyzed=len(files_analyzed),
                quality_score=85,  # Placeholder
                grade="B",  # Placeholder
                recommendations=recommendations,
                summary={
                    "quality_score": 85,
                    "grade": "B",
                    "issues_found": len(recommendations)
                }
            ).dict()
            
        except Exception as e:
            return {"error": f"Code quality analysis failed: {str(e)}"}